<a href="https://colab.research.google.com/github/ElimuMichael/TensorFlow/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchvision

    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 591.8MB 21kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6238c000 @  0x7f1d8dc2f2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 2.0MB 4.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [17]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#@title
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
from google.colab import files, drive
# This will prompt for authorization.
drive.mount('/content/drive')
drive_path = "/content/drive/My Drive/"
file_path = drive_path + 'LSTM/data/anna.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [20]:
# Load data
with open(file_path, 'r') as data:
    text = data.read()

# Tokenize the words
chars = tuple(set(text))

# print(chars)
int2chars = dict(enumerate(chars))
# print(int2chars)
chars2int = {chx: i for i, chx in int2chars.items()}
# print(chars2int)

# Encoded text
encoded = np.array([chars2int[ch] for ch in text])
print(encoded[:100])

[76  4 81 40 68 53 59 63 36 43 43 43 61 81 40 40 78 63 18 81 64  1 39  1
 53 34 63 81 59 53 63 81 39 39 63 81 39  1 20 53  3 63 53 46 53 59 78 63
 33 11  4 81 40 40 78 63 18 81 64  1 39 78 63  1 34 63 33 11  4 81 40 40
 78 63  1 11 63  1 68 34 63 19 54 11 43 54 81 78 79 43 43 21 46 53 59 78
 68  4  1 11]


In [0]:
# Preprocess the data
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype="float32")
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot
# Test the one_hot encode
# print(one_hot_encode(np.array([[1, 2, 6]]), 7))

# Generate training mini-batches
# Discard all the values that don't fit into the sequence length window
# Each window is batch_size X sequence_length

In [22]:
def get_batches(arr, batch_size, seq_length):
    '''
    Create a generator that returns batches of size
    batch_size x seq_length
    '''
    # total of the batch size
    batch_size_total = batch_size* seq_length
    
    # Number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Characters making full batches
    arr = arr[:n_batches*batch_size_total]
    # Reshape the array into batch_size rows
    arr = arr.reshape((batch_size, -1))

    # Iterate over the batches using the window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # Extract the features
        x = arr[:, n:n+seq_length]
        # print(x)
        # Targets shifted by one
        y = np.zeros_like(x)
        # print(y)
        # Generator by lstting the value of y equals the value of
        # of x shifted by one
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            # Make the last value of y equal to the first value of x in the sequence
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# batches = get_batches(encoded, 8, 50)
# x, y = next(batches)
# print('x\n', x[:10, :10])
# print('y\n', y[:10, :10])

# Define the model
# Check GPU's availability
train_on_GPU = torch.cuda.is_available()
if train_on_GPU:
    print('Training on GPU')
else:
    print('No GPU available, training on CPU; consider making n_epochs smaller.')

No GPU available, training on CPU; consider making n_epochs smaller.


In [0]:
class CharRNN(nn.Module):
    """docstring for CharRNN"""
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.01):
        super(CharRNN, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        # Obtain the output and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_GPU):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10,batch_size=10, seq_length=50, lr=0.01, clip=5, val_frac=0.2, print_every=10):
    # Allow for training and use of the drop out
    net.train()

    # Define the optimizer and the loss function    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_GPU):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_GPU):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            targets = targets.long()
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_GPU):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)

                    targets = targets.long()

                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

# Instantiate the model
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)
Epoch: 1/20... Step: 10... Loss: 3.2148... Val Loss: 3.1818


In [0]:
# Making Predictions
def predict(net, char, h=None, top_k=None):
    '''
    Given a character, predict the next character.
    Returns the predicted character and the hidden state
    '''
    # Tensor inputs
    x = np.array([[net.chars2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if train_on_GPU:
        inputs = inputs.cuda()

    # Detachh hidden state from the history
    h = tuple([each.data for each in h])

    # Get the model output
    out, h = net(inputs, h)

    # Obtain the character probabilities
    p = F.softmax(out, dim=1).data

    if train_on_GPU:
        p = p.cpu() # Move the results to the CPU

    # Get the top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))

    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # Select the likely next character with some level of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())

    # Return the encoded value of the predicted character and hidden state
    return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
    if train_on_GPU:
        net.cuda()

    else:
        net.cpu()

    chars = [ch for ch in prime]
    h = net.init_hidden(1)

    for ch in prime:
        char, h = predict(net, chars[-1], h, top_k=top_k)

    chars.append(char)

    # Pass in the previous character and get a new one
    for i in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
print(sample(net, 1000, prime='Anna', top_k=5))

# Save the model checkpoint
model_name = 'rnn_20_epoch.net'

checkpoint = {
    'n_hidden': net.n_hidden,
    'n_layers': net.n_layers,
    'state_dict': net.state_dict,
    'tokens': net.chars
}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

# Loading Checkpoint
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'],n_hidden=checkpoint['hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

print(sample(net, 2000, prime='And Michael Said', top_k=5))